<a href="https://colab.research.google.com/github/navavatpi/researchPhd/blob/developModeling/dataModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # https://pypi.python.org/pypi/pydot
# !apt-get -qq install -y graphviz && pip install pydot
# import pydot

# Template

## Environment Setup

In [2]:
# Enabling and testing the TPU
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.6.0
Running on TPU  ['10.22.182.90:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.22.182.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.182.90:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [12]:
# libraries import
import os
import numpy as np
from tensorflow import keras # back-end artificial neural network API
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam # optimizer
from tensorflow.keras.callbacks import ReduceLROnPlateau # Reduce learning rate when a metric has stopped improving

# system configuration
os.environ["TF_CPP_MIN_LOG_LEVEL"]="0" # disable warning messages
os.environ['AUTOGRAPH_VERBOSITY'] = '0' # set it to 0

random_seed = 20 # random seed

In [4]:
# connect to google drive to obtain dataset
# !note: need to perform authorization by following below link
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Loading

In [5]:
# define path to files
defaultdir = '/content/drive/My Drive/all_about_phd/'

In [6]:
# save processed datas into drive
X_train = np.load(defaultdir + '/processed_data/X_train.npy')
X_test = np.load(defaultdir + '/processed_data/X_test.npy')
y_train = np.load(defaultdir + '/processed_data/y_train.npy')
y_test = np.load(defaultdir + '/processed_data/y_test.npy')

## Data Modeling

In [14]:
# define function to create CNN model
def cnnModel(data_train, num_classes, layer=1, x_filter= 64, 
             kernel_size= (3,3), pool_size=(2,2), drop_size=0.5, 
             ):
    
    # input layer convert arrays => tensors
    inputs = keras.Input(data_train.shape[1:]) # dimension of the input

    x1 = layers.Conv2D(filters = x_filter, 
                       kernel_size = kernel_size, 
                       padding = 'Same', 
                       activation ='relu')(inputs)
    x1 = layers.Conv2D(filters = x_filter, 
                       kernel_size = kernel_size, 
                       padding = 'Same', 
                       activation ='relu')(x1)
    x1 = layers.MaxPool2D(pool_size= pool_size)(x1)
    x1 = layers.Dropout(drop_size)(x1)
    
    dense_size = x_filter
    
    if layer > 1:
        
        x1 = layers.Conv2D(filters = 2*x_filter, 
                           kernel_size = kernel_size, 
                           padding = 'Same', 
                           activation ='relu')(x1)
        x1 = layers.Conv2D(filters = 2*x_filter, 
                           kernel_size = kernel_size, 
                           padding = 'Same', 
                           activation ='relu')(x1)
        x1 = layers.MaxPool2D(pool_size= pool_size)(x1)
        x1 = layers.Dropout(drop_size)(x1)
        
        dense_size = 2*x_filter
        
        if layer > 2:

            x1 = layers.Conv2D(filters = 4*x_filter, 
                               kernel_size = kernel_size, 
                               padding = 'Same', 
                               activation ='relu')(x1)
            x1 = layers.Conv2D(filters = 4*x_filter, 
                               kernel_size = kernel_size, 
                               padding = 'Same', 
                               activation ='relu')(x1)
            x1 = layers.MaxPool2D(pool_size= pool_size)(x1)
            x1 = layers.Dropout(drop_size)(x1)
            
            dense_size = 4*x_filter
            
            if layer > 3:

                x1 = layers.Conv2D(filters = 8*x_filter, 
                                   kernel_size = kernel_size, 
                                   padding = 'Same', 
                                   activation ='relu')(x1)
                x1 = layers.Conv2D(filters = 8*x_filter, 
                                   kernel_size = kernel_size, 
                                   padding = 'Same', 
                                   activation ='relu')(x1)
                x1 = layers.MaxPool2D(pool_size= pool_size)(x1)
                x1 = layers.Dropout(drop_size)(x1)
                
                dense_size = 8*x_filter
                
                if layer > 4:

                    x1 = layers.Conv2D(filters = 8*x_filter, 
                                       kernel_size = kernel_size, 
                                       padding = 'Same', 
                                       activation ='relu')(x1)
                    x1 = layers.Conv2D(filters = 8*x_filter, 
                                       kernel_size = kernel_size, 
                                       padding = 'Same', 
                                       activation ='relu')(x1)
                    x1 = layers.MaxPool2D(pool_size= pool_size)(x1)
                    x1 = layers.Dropout(drop_size)(x1)
                    
                    dense_size = 8*x_filter

    x2 = layers.Flatten()(x1)
    x2 = layers.Dense(dense_size, activation = "relu")(x2)

    outputs = layers.Dense(num_classes, activation= "softmax")(x2)

    cnnModel = keras.Model(inputs=inputs, outputs=outputs, name="CNN_model")

    cnnModel.summary()
    
    return cnnModel

In [15]:
# perform CNN model define function
cnnModel = cnnModel(X_train, layer=2, num_classes=3)

Model: "CNN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 330, 640, 224)]   0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 330, 640, 64)      129088    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 330, 640, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 165, 320, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 165, 320, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 165, 320, 128)     73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 165, 320, 128)     14